# Collaborative filtering using WALS algorithm

## Create raw dataset

In [16]:
PROJECT = 'qwiklabs-gcp-4a684069c4776675'
BUCKET = 'colaborative-filtering-agea'
REGION = 'us-central1'

In [3]:
import tensorflow as tf
from google.cloud import bigquery

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [0]:
from google.colab import auth
auth.authenticate_user()

In [6]:
sql = """
#standardSQL
SELECT 
  user_id as visitorId,
  content_id as contentId,
  COUNT(user_id) as numberOfVisits

  FROM AGEA_ASL.Dataset_B
  GROUP BY contentId,visitorId
"""

In [ ]:
#df = client.query(sql + 'LIMIT 1000').to_dataframe() # TODO: SACAR LIMIT
#df.head()

In [7]:
import google.datalab.bigquery as bq
df = bq.Query(sql).execute().result().to_dataframe()
df.head()

,visitorId,contentId,numberOfVisits
0,3127735,JVAFAfO5p,1
1,1402547,YhYp5WCm-,1
2,4424328,ekUWVdEXY,1
3,3604872,Y-vZd0XoQ,1
4,3627874,2nvx6mPGW,1


In [8]:
stats = df.describe()

In [ ]:
# the rating is the numberOfVisits scaled to be in the range 0-1
median = stats['numberOfVisits']['50%']
df['rating'] = 0.3 * df['numberOfVisits'] / median
df.loc[df['rating'] > 1, 'rating'] = 1

In [ ]:
del df['numberOfVisits']

In [13]:
!rm -rf data
!mkdir data

In [14]:
df.to_csv('data/collab_raw.csv', index=False, header=False)

In [17]:
# upload to bucket just in case that the local csv be erased
!gsutil cp 'data/collab_raw.csv'  gs://{BUCKET}/collab_raw.csv 

Copying file://data/collab_raw.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

| [1 files][835.9 MiB/835.9 MiB]      0.0 B/s                                   
Operation completed over 1 objects/835.9 MiB.                                    


In [18]:
!head data/collab_raw.csv

3127735,JVAFAfO5p,0.3
1402547,YhYp5WCm-,0.3
4424328,ekUWVdEXY,0.3
3604872,Y-vZd0XoQ,0.3
3627874,2nvx6mPGW,0.3
3862440,9P5DtZj9t,0.3
4961169,0bMjAUf8i,0.3
3853869,SRlruhYHQ,0.3
4491186,ws7UTZTlf,0.3
5604673,5wAUEiwRI,0.3
